In [ ]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install bayesian-optimization

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import math
import pickle
import time
import sys

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.losses import mse
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from bayes_opt import BayesianOptimization

%matplotlib inline

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

# BoT IoT Data Pre-Processing

In [ ]:
# Traffic information

path = '../botiot/'
all_files = glob.glob(os.path.join(path , '*.csv'))

files_list = []
for file in all_files:
    df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
    files_list.append(df)

df_traffic = pd.concat(files_list, axis = 0, ignore_index = True)
df_traffic = df_traffic.drop(columns=['pkSeqID', 'stime', 'flgs', 'flgs_number', 'saddr', 'sport', 'daddr', 'dport', 'subcategory'])
df_traffic['proto'] = df_traffic['proto'].map({'tcp': 1, 'arp': 2, 'udp': 3, 'icmp': 4, 'ipv6-icmp': 5})
df_traffic['state'] = df_traffic['state'].map({'REQ': 1, 'RST': 2, 'ACC': 3, 'CON': 4, 'INT': 5, 'URP': 6, 'FIN': 7, 'NRS': 8, 'ECO': 9, 'TST': 10, 'MAS': 11})

In [ ]:
# Attacks

df_attack = df_traffic.query('attack == 1')
df_attack = df_attack.drop(columns=['attack'])
df_attack['category'] = df_attack['category'].map({'DDoS': 0, 'DoS': 1, 'Reconnaissance': 2, 'Theft': 3})

df_ddos = df_attack.query('category == 0')
label_ddos = df_ddos.pop('category')

df_dos = df_attack.query('category == 1')
label_dos = df_dos.pop('category')

df_rec = df_attack.query('category == 2')
label_rec = df_rec.pop('category')

df_theft = df_attack.query('category == 3')
label_theft = df_theft.pop('category')

In [ ]:
scaler = MinMaxScaler()

df_ddos_norm = scaler.fit_transform(df_ddos)
df_dos_norm = scaler.fit_transform(df_dos)
df_rec_norm = scaler.fit_transform(df_rec)
df_theft_norm = scaler.fit_transform(df_theft)

# CNN - Attack Classification

In [ ]:
batch_size = 50
number_features = 23
learning_rate = 0.008
epochs = 25

dict_params = { 'learning_rate': learning_rate, 'batch_size': int(32 * batch_size), 'epochs': int(epochs) }
pbounds = { 'learning_rate': (0.000001, 0.1), 'batch_size': (1, 4.001), 'epochs': (1, 50) }

In [ ]:
# Train set

len_ddos_train = int(0.7 * len(df_ddos_norm))
X_ddos_train = df_ddos_norm[:len_ddos_train]

len_dos_train = int(0.7 * len(df_dos_norm))
X_dos_train = df_dos_norm[:len_dos_train]

len_rec_train = int(0.7 * len(df_rec_norm))
X_rec_train = df_rec_norm[:len_rec_train]

len_theft_train = int(0.7 * len(df_theft_norm))
X_theft_train = df_theft_norm[:len_theft_train]

np_train = np.concatenate([X_ddos_train, X_dos_train, X_rec_train, X_theft_train])
np_label_train = np.concatenate([label_ddos[:len_ddos_train], label_dos[:len_dos_train], label_rec[:len_rec_train], 
                                 label_theft[:len_theft_train]])

df_train = pd.DataFrame(np_train)
df_label_train = pd.DataFrame(np_label_train)

X_train = df_train.to_numpy()
Y_train = df_label_train.to_numpy()

print(len(X_train))

In [ ]:
# Test set

len_ddos_test = len_ddos_train + int(0.15 * len(df_ddos_norm))
X_ddos_test = df_ddos_norm[len_ddos_train : len_ddos_test]

len_dos_test = len_ddos_train + int(0.15 * len(df_dos_norm))
X_dos_test = df_dos_norm[len_dos_train : len_dos_test]

len_rec_test = len_rec_train + int(0.15 * len(df_rec_norm))
X_rec_test = df_rec_norm[len_rec_train : len_rec_test]

len_theft_test = len_theft_train + int(0.15 * len(df_theft_norm))
X_theft_test = df_theft_norm[len_theft_train : len_theft_test]

np_test = np.concatenate([X_ddos_test, X_dos_test, X_rec_test, X_theft_test])
np_label_test = np.concatenate([label_ddos[len_ddos_train : len_ddos_test], label_dos[len_dos_train : len_dos_test], 
                                 label_rec[len_rec_train : len_rec_test], label_theft[len_theft_train : len_theft_test]])

df_test = pd.DataFrame(np_test)
df_label_test = pd.DataFrame(np_label_test)

X_test = df_test.to_numpy()
Y_test = df_label_test.to_numpy()

print(len(X_test))

In [ ]:
# Validation set

X_ddos_val = df_ddos_norm[len_ddos_test:]
X_dos_val = df_dos_norm[len_dos_test:]
X_rec_val = df_rec_norm[len_rec_test:]
X_theft_val = df_theft_norm[len_theft_test:]

np_val = np.concatenate([X_ddos_val, X_dos_val, X_rec_val, X_theft_val])
np_label_val = np.concatenate([label_ddos[len_ddos_test:], label_dos[len_dos_test:], label_rec[len_rec_test:], 
                               label_theft[len_theft_test:]])

df_val = pd.DataFrame(np_val)
df_label_val = pd.DataFrame(np_label_val)

X_val = df_val.to_numpy()
Y_val = df_label_val.to_numpy()

print(len(X_val))

In [ ]:
X_train_CNN = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_CNN = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val_CNN = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

Y_train_CNN = Y_train
Y_test_CNN = Y_test
Y_val_CNN = Y_val

samples, feature, depth = X_train_CNN.shape

In [ ]:
# CNN model with 1D convolutional layer

def CNN():
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform', input_shape=(feature, depth)))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    return model

In [ ]:
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

## Train

In [ ]:
# Training step with Bayesian optimization

def training(X_train = X_train_CNN,
             Y_train = Y_train_CNN, 
             X_val = X_val_CNN, 
             Y_val = Y_val_CNN, 
             X_test = X_test_CNN, 
             Y_test = Y_test_CNN, 
             learning_rate = learning_rate, 
             epochs = epochs, 
             batch_size = batch_size,
             reduce_lr = reduce_lr):
    
    nadam = optimizers.Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
    
    model = CNN()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
    
    history = model.fit(X_train, Y_train, 
                        epochs = dict_params['epochs'], 
                        batch_size = dict_params['batch_size'], 
                        validation_data = (X_val, Y_val),
                        callbacks = [reduce_lr])
    
    scores = model.evaluate(X_test, Y_test)
    
    print('loss:', scores[0])
    print('accuracy:', scores[1])
    return scores[1]

In [ ]:
"""
# Bayesian optimization to choose the best hyperparameters

optimizer = BayesianOptimization(
    f = training,
    pbounds = pbounds,
    verbose = 2, 
    random_state = 1,
)

train_start = time.time()

optimizer.maximize(init_points = 5, n_iter = 5)

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

print(optimizer.max)
"""

In [ ]:
# Training step with the best hyperparameters

nadam = optimizers.Nadam(learning_rate = 0.01946960847019594, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)

model = CNN()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])

train_start = time.time()

history = model.fit(X_train_CNN, Y_train_CNN, 
                    epochs = int(36.701988678683904), 
                    batch_size = int(32 * 3.179494411695542), 
                    validation_data = (X_val_CNN, Y_val_CNN),
                    callbacks = [reduce_lr])

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

## Test

In [ ]:
# Testing step

test_start = time.time()

Y_pred = model.predict(X_test_CNN)

test_end = time.time()
test_time = test_end - test_start
print("Testing time:", test_time)

In [ ]:
Y_pred_CNN = np.argmax(Y_pred, axis = 1)
Y_true_CNN = Y_test_CNN.astype(int)

## Metrics

In [ ]:
labels = ['DDoS', 'DoS', 'Reconaissance', 'Theft']

In [ ]:
# Multi classification metrics

acc = accuracy_score(Y_true_CNN, Y_pred_CNN) 
f1 = f1_score(Y_true_CNN, Y_pred_CNN, average = 'weighted')
pre = precision_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted')
recall = recall_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

In [ ]:
sys.stdout = open("../Results/bot_iot_cnn.txt", "a")

print(" ==== Test " + str(number_features) + " features ====")
print("Training time:" + str(train_time))
print("Testing time:" + str(test_time))
print("Accuracy:" + str(acc))
print("F1-score:" + str(f1))
print("Precision:" + str(pre))
print("Recall:" + str(recall))
#print("Hyper-parameters:" + str(optimizer.max))
print(classification_report(Y_true_CNN,Y_pred_CNN, target_names = labels))

# Naive Bayes

In [ ]:
# Model
nb = GaussianNB()

# Train
train_nb_start = time.time()

nb.fit(X_train, Y_train)

train_nb_end = time.time()
train_nb_time = train_nb_end - train_nb_start

# Test
test_nb_start = time.time()

Y_pred_nb = nb.predict(X_test)

test_nb_end = time.time()
test_nb_time = test_nb_end - test_nb_start

# Metrics
acc_nb = accuracy_score(Y_test, Y_pred_nb) 
f1_nb = f1_score(Y_test, Y_pred_nb, average = 'weighted')
pre_nb = precision_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted')
recall_nb = recall_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== Naive Bayes " + str(number_features) + " features ====")
print("Training time:" + str(train_nb_time))
print("Testing time:" + str(test_nb_time))
print("Accuracy:" + str(acc_nb))
print("F1-score:" + str(f1_nb))
print("Precision:" + str(pre_nb))
print("Recall:" + str(recall_nb))
print(classification_report(Y_test,Y_pred_nb, target_names = labels))

# KNN

In [ ]:
# Model
knn = KNeighborsClassifier(n_neighbors = 50)

# Train
train_knn_start = time.time()

knn.fit(X_train, Y_train)

train_knn_end = time.time()
train_knn_time = train_knn_end - train_knn_start

# Test
test_knn_start = time.time()

Y_pred_knn = knn.predict(X_test)

test_knn_end = time.time()
test_knn_time = test_knn_end - test_knn_start

# Metrics
acc_knn = accuracy_score(Y_test, Y_pred_knn) 
f1_knn = f1_score(Y_test, Y_pred_knn, average = 'weighted')
pre_knn = precision_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted')
recall_knn = recall_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== KNN " + str(number_features) + " features ====")
print("Training time:" + str(train_knn_time))
print("Testing time:" + str(test_knn_time))
print("Accuracy:" + str(acc_knn))
print("F1-score:" + str(f1_knn))
print("Precision:" + str(pre_knn))
print("Recall:" + str(recall_knn))
print(classification_report(Y_test,Y_pred_knn, target_names = labels))